In [1]:
import re
import os
import numpy as np
from progress.bar import Bar
from tqdm import tqdm
import time

In [4]:

def writeRadius(r1,r2):
    lpMesh = open("loop.geo", "r")
    loopMesh = lpMesh.read()
    lpMesh.close()

    reg1 = r"(%r1)"
    reg2 = r"(%r2)"

    sub1 = str(r1)
    sub2 = str(r2)

    loopMesh = re.sub(reg1,sub1,loopMesh,0,re.MULTILINE)
    loopMesh = re.sub(reg2,sub2,loopMesh,0,re.MULTILINE)

    try:
        lpTemp = open("loopTemp.geo", "x")
    except:
        os.remove("loopTemp.geo")
        lpTemp = open("loopTemp.geo","x")

    lpTemp.write(loopMesh)
    lpTemp.close()


    # write surface mesh
    lpMesh = open("loopFIELD.geo", "r")
    loopMesh = lpMesh.read()
    lpMesh.close()
    reg1 = r"(%r1)"
    reg2 = r"(%r2)"
    sub1 = str(r1)
    sub2 = str(r2)
    loopMesh = re.sub(reg1,sub1,loopMesh,0,re.MULTILINE)
    loopMesh = re.sub(reg2,sub2,loopMesh,0,re.MULTILINE)
    try:
        lpTemp = open("loopFIELDTemp.geo", "x")
    except:
        os.remove("loopFIELDTemp.geo")
        lpTemp = open("loopFIELDTemp.geo","x")
    lpTemp.write(loopMesh)
    lpTemp.close()

def runGMSH():
    print(f"{os.getpid()}: Meshing geometry\n")
    # os.system("gmsh loopFIELDTemp.geo -nopopup -0 -v 1")
    # os.system("gmsh loopTemp.geo -nopopup -0 -v 1")
    os.popen("gmsh loopFIELDTemp.geo -nopopup -0 -v 1").read()
    os.popen("gmsh loopTemp.geo -nopopup -0 -v 1").read()
    try:
        os.system("rm *.geo_unrolled")
        os.remove("loopFIELDTemp.geo")
        os.remove("loopTemp.geo")
    except:
        print("nope")
    print(f"{os.getpid()}: Done meshing\n")


def runTTH():
    print(f"{os.getpid()}: Running tth settings_washer.json >/dev/null 2>&1")
    os.system("tth settings_washer.json >/dev/null 2>&1")
    print(f"{os.getpid()}: Done\n")


def getCurrent():
    iFile = open("./output/i.txt")
    it = iFile.read()
    iFile.close()
    reg = r"fone fone .+ (\d*\.\d*e.\d*)"
    i_s = re.findall(reg,it,re.MULTILINE)[0]
    return i_s
    
def runNoiseEx(b_path,current):
    path = "/home/paul/Skripsie/Code/SkripsieCode/cpp/noiseEX/cmake-build-debug/nex"
    print(f"{os.getpid()}: Running "+path+" "+b_path+" "+current)
    v = os.popen(path+" "+b_path+" "+current).read()
    print(f"{os.getpid()}: Done\n")
    v = v.split("\n")
    return v

def getElementsAndNodeCount():
    iFile = open("./loop.msh")
    it = iFile.read()
    iFile.close()
    regNumNodes = r"\$Nodes\n(.*)\n"
    numNodes = re.findall(regNumNodes,it,re.MULTILINE)[0]
    regNumElem = r"\$Elements\n(.*)\n"
    numElem = re.findall(regNumElem,it,re.MULTILINE)[0]

    return numElem, numNodes



### Triangles:

In [6]:
os.chdir("/home/paul/Skripsie/Code/SkripsieCode/cpp/noiseEX/testfiles/testTriangularMeshing/triangles/")

SPIN_DENS = 5e17
MU_B = 9.2740100783e-24
PHI_0 = 2.067833848e-15
MU_0 = 1.25663706212e-6 	
Analytic = lambda R, D : 2/3*MU_0**2*MU_B**2*SPIN_DENS*R/D*1/(PHI_0**2)

r1s = np.linspace(15e-6, 30e-6, 5)
r2 = 0.2e-6


try:
    os.remove("triangles.csv")
except:
    pass

res = open("triangles.csv","w")
res.write("R,D,Analytic,Numerical,Analytic Area,Numerical Area,Mesh Time,TTH Time,nex Time,Num Elements,Num Nodes\n")

for r1 in tqdm(r1s):
    writeRadius(r1, r2)
    start = time.time()
    runGMSH()
    end = time.time()
    meshTime = end - start

    numElem , numNodes = getElementsAndNodeCount()

    start = time.time()
    runTTH()
    end = time.time()
    tthTime = end - start
    i = getCurrent()
    print(i)
    start = time.time()
    msfn_numerical = runNoiseEx("/home/paul/Skripsie/Code/SkripsieCode/cpp/noiseEX/testfiles/testTriangularMeshing/triangles/output/loop_test.vtk",i)
    end = time.time()
    nexTime = end - start
    numerical_area = msfn_numerical[1]
    msfn_numerical = msfn_numerical[0]
    msfn_analytical = Analytic(r1,r2*2)
    analytic_area = 4*np.pi**2*r1*r2
    # write result to file
    res.write(f"{r1},{2*r2},{msfn_analytical},{msfn_numerical},{analytic_area},{numerical_area},{meshTime},{tthTime},{nexTime},{numElem},{numNodes}\n")



res.close()

  0%|          | 0/5 [00:00<?, ?it/s]

27992: Meshing geometry

27992: Done meshing

27992: Running tth settings_washer.json >/dev/null 2>&1
27992: Done

1.4660638549e-01
27992: Running /home/paul/Skripsie/Code/SkripsieCode/cpp/noiseEX/cmake-build-debug/nex /home/paul/Skripsie/Code/SkripsieCode/cpp/noiseEX/testfiles/testTriangularMeshing/triangles/output/loop_test.vtk 1.4660638549e-01


 20%|██        | 1/5 [00:20<01:20, 20.07s/it]

27992: Done

27992: Meshing geometry

27992: Done meshing

27992: Running tth settings_washer.json >/dev/null 2>&1
27992: Done

1.1281031481e-01
27992: Running /home/paul/Skripsie/Code/SkripsieCode/cpp/noiseEX/cmake-build-debug/nex /home/paul/Skripsie/Code/SkripsieCode/cpp/noiseEX/testfiles/testTriangularMeshing/triangles/output/loop_test.vtk 1.1281031481e-01


 40%|████      | 2/5 [00:51<01:20, 26.82s/it]

...

27992: Done

27992: Meshing geometry

27992: Done meshing

27992: Running tth settings_washer.json >/dev/null 2>&1
27992: Done

7.6322413658e-02
27992: Running /home/paul/Skripsie/Code/SkripsieCode/cpp/noiseEX/cmake-build-debug/nex /home/paul/Skripsie/Code/SkripsieCode/cpp/noiseEX/testfiles/testTriangularMeshing/triangles/output/loop_test.vtk 7.6322413658e-02


 80%|████████  | 4/5 [02:33<00:43, 43.98s/it]

27992: Done

27992: Meshing geometry

27992: Done meshing

27992: Running tth settings_washer.json >/dev/null 2>&1
27992: Done

6.5405158233e-02
27992: Running /home/paul/Skripsie/Code/SkripsieCode/cpp/noiseEX/cmake-build-debug/nex /home/paul/Skripsie/Code/SkripsieCode/cpp/noiseEX/testfiles/testTriangularMeshing/triangles/output/loop_test.vtk 6.5405158233e-02


100%|██████████| 5/5 [03:50<00:00, 46.20s/it]

27992: Done



### Tetras:

In [7]:
os.chdir("/home/paul/Skripsie/Code/SkripsieCode/cpp/noiseEX/testfiles/testTriangularMeshing/tetra/")

SPIN_DENS = 5e17
MU_B = 9.2740100783e-24
PHI_0 = 2.067833848e-15
MU_0 = 1.25663706212e-6 	
Analytic = lambda R, D : 2/3*MU_0**2*MU_B**2*SPIN_DENS*R/D*1/(PHI_0**2)

r1s = np.linspace(15e-6, 30e-6, 5)
r2 = 0.2e-6


try:
    os.remove("tetras.csv")
except:
    pass

res = open("tetras.csv","w")
res.write("R,D,Analytic,Numerical,Analytic Area,Numerical Area,Mesh Time,TTH Time,nex Time,Num Elements,Num Nodes\n")

for r1 in tqdm(r1s):
    writeRadius(r1, r2)
    start = time.time()
    runGMSH()
    end = time.time()
    meshTime = end - start

    numElem , numNodes = getElementsAndNodeCount()

    start = time.time()
    runTTH()
    end = time.time()
    tthTime = end - start
    i = getCurrent()
    print(i)
    start = time.time()
    msfn_numerical = runNoiseEx("/home/paul/Skripsie/Code/SkripsieCode/cpp/noiseEX/testfiles/testTriangularMeshing/tetra/output/loop_test.vtk",i)
    end = time.time()
    nexTime = end - start
    numerical_area = msfn_numerical[1]
    msfn_numerical = msfn_numerical[0]
    msfn_analytical = Analytic(r1,r2*2)
    analytic_area = 4*np.pi**2*r1*r2
    # write result to file
    res.write(f"{r1},{2*r2},{msfn_analytical},{msfn_numerical},{analytic_area},{numerical_area},{meshTime},{tthTime},{nexTime},{numElem},{numNodes}\n")



res.close()

os.chdir("/home/paul/Skripsie/Code/SkripsieCode/cpp/noiseEX/testfiles/testTriangularMeshing/")

  0%|          | 0/5 [00:00<?, ?it/s]

27992: Meshing geometry

27992: Done meshing

27992: Running tth settings_washer.json >/dev/null 2>&1
27992: Done

1.0943227870e-01
27992: Running /home/paul/Skripsie/Code/SkripsieCode/cpp/noiseEX/cmake-build-debug/nex /home/paul/Skripsie/Code/SkripsieCode/cpp/noiseEX/testfiles/testTriangularMeshing/tetra/output/loop_test.vtk 1.0943227870e-01


 20%|██        | 1/5 [00:32<02:09, 32.29s/it]

27992: Done

27992: Meshing geometry

27992: Done meshing

27992: Running tth settings_washer.json >/dev/null 2>&1
27992: Done

8.5015196732e-02
27992: Running /home/paul/Skripsie/Code/SkripsieCode/cpp/noiseEX/cmake-build-debug/nex /home/paul/Skripsie/Code/SkripsieCode/cpp/noiseEX/testfiles/testTriangularMeshing/tetra/output/loop_test.vtk 8.5015196732e-02


 40%|████      | 2/5 [01:20<02:04, 41.42s/it]

...

27992: Done

27992: Meshing geometry

27992: Done meshing

27992: Running tth settings_washer.json >/dev/null 2>&1
27992: Done

5.8251250161e-02
27992: Running /home/paul/Skripsie/Code/SkripsieCode/cpp/noiseEX/cmake-build-debug/nex /home/paul/Skripsie/Code/SkripsieCode/cpp/noiseEX/testfiles/testTriangularMeshing/tetra/output/loop_test.vtk 5.8251250161e-02


 80%|████████  | 4/5 [03:46<01:03, 63.76s/it]

27992: Done

27992: Meshing geometry

27992: Done meshing

27992: Running tth settings_washer.json >/dev/null 2>&1
27992: Done

5.0158929349e-02
27992: Running /home/paul/Skripsie/Code/SkripsieCode/cpp/noiseEX/cmake-build-debug/nex /home/paul/Skripsie/Code/SkripsieCode/cpp/noiseEX/testfiles/testTriangularMeshing/tetra/output/loop_test.vtk 5.0158929349e-02


100%|██████████| 5/5 [05:26<00:00, 65.30s/it]

27992: Done



In [5]:
writeRadius(15e-6,0.2e-6)